In [1]:
# !pip install optuna

In [2]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import optuna
import io

In [3]:
# path to the dataset (replace with your own path) MAKE SURE YOU CHANGE THE SLASHES
path = "C:/Users/Asa/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1"

In [4]:
import preprocessing as prep
target_size = (150, 150)
grayscale = True
X, y = prep.get_X_y(path=path, percent=0.25, target_size=target_size, grayscale=grayscale)

In [5]:
no_output_classes = len(y.columns)

In [6]:
y[:5]

,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X.shape

(1360, 150, 150)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 1020 1020
Test Lengths: 340 340


In [10]:
# defining the model
from tensorflow.keras import layers

input_shape = target_size
if grayscale:
    input_shape = input_shape + (1,)
else:
    input_shape = input_shape + (3,)

# removing images from the dataset
# dropout: randomly remove nodes in each layer to create gaps
# normalization layer
# augmentation
# convert to grayscale <- 


# creating the create_model function for optuna
def create_model(trial):
    n_layers = trial.suggest_int('n_layers', 1, 5)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    
    model = keras.Sequential()
    global input_shape
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))

    for i in range(n_layers):
        # add convolutional layer
        num_units = trial.suggest_int(f'n_units_{i}', 32, 128, step=32)
        model.add(layers.Conv2D(num_units, (3, 3), activation='relu'))

        # if last layer, additionaly add a batch normalization layer
        if i == n_layers - 1:
            model.add(layers.BatchNormalization())

        # add pooling layer
        model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dropout(dropout_rate))

    global no_output_classes
    model.add(layers.Dense(no_output_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
# defining the objective function for optuna
def objective(trial):
    model = create_model(trial)

    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    score = model.evaluate(X_test, y_test, verbose=1)
    return score[1]

In [ ]:
# running optimization with optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-04-07 21:33:48,932] A new study created in memory with name: no-name-38a39f24-a0f4-4ada-9365-23a026edb053
c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-04-07 21:36:23,986] Trial 0 finished with value: 0.33529412746429443 and parameters: {'n_layers': 5, 'dropout_rate': 0.11200679826456393, 'n_units_0': 64, 'n_units_1': 128, 'n_units_2': 96, 'n_units_3': 64, 'n_units_4': 64}. Best is trial 0 with value: 0.33529412746429443.
c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as th

In [ ]:
# creating the model
model = create_model(study.best_trail)

In [42]:
# Evaluate the model using the testing data
model.evaluate(X_test, y_test)

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.4561 - loss: 1.5874


[1.6147503852844238, 0.44938650727272034]

In [43]:
# Saving the model
with open("initial_model.pkl", "wb") as f:
    pickle.dump(model, f)

### Testing the model

In [44]:
# reloaing the model
import pickle
with open("initial_model.pkl", "rb") as f:
    model = pickle.load(f)

In [45]:
# preprocess a new image 
import preprocessing as prep
image_path ="clothes_test_image.jpg"
X_input = prep.get_X([image_path], target_size=(150, 150))

In [46]:
import pandas as pd
output = model.predict(X_input)
pd.DataFrame(output, columns=prep.encoded_y_cols)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_3
Received: inputs=('Tensor(shape=(1, 150, 150))', ())
  warnings.warn(msg)


,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,0.96198,0.461258,0.997945,0.684569,0.696039,0.385976,0.654634,0.036925


In [47]:
import numpy as np
predicted_class = np.argmax(output[0])
prep.encoded_y_cols[predicted_class]

'clothes'

In [48]:
prep.get_prediction(image_path=image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_3
Received: inputs=('Tensor(shape=(1, 150, 150))', ())
  warnings.warn(msg)


'clothes'